In [1]:
!pip install -q --upgrade --user google-cloud-aiplatform google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.1/240.1 kB 23.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}
print(PROJECT_ID)

qwiklabs-gcp-00-2ac1f74fb415


In [3]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print("Initialized")

Initialized


In [4]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

def answer_question_gemini(prompt):
  model = GenerativeModel("gemini-pro")
  response = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 8192,
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 10,
    },
  stream=False,
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Cleaning the Data")
    return "An Error Ocuured Cleaning the Data"

In [5]:
def run_search(question):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = """
      SELECT query.query, base.title, base.body
      FROM VECTOR_SEARCH(
      TABLE `bbc_news.bbc_news_with_embeddings_table`, 'text_embedding',
      (
      SELECT text_embedding, content AS query
      FROM ML.GENERATE_TEXT_EMBEDDING(MODEL `bbc_news.embedding_model`,
          (SELECT @question AS content))),
      top_k => 5)
      """

  job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("question", "STRING", question),
    ]
  )

  query_job = client.query(sql, job_config=job_config)

  data = ""
  for row in query_job:
    data += row.body + "\n"

  return data

In [6]:
def build_prompt(data, question):
  prompt = """
    Instructions: Answer the question using the following Context.

    Context: {0}

    Question: {1}
  """.format(data, question)
  return prompt

In [7]:
from IPython.core.display import display, HTML

def answer_question(question):

  data = run_search(question)
  display("Retrieved Data:")
  display(data)
  display(" . . . ")
  prompt = build_prompt(data, question)
  answer_gemini = answer_question_gemini(prompt)

  return answer_gemini

In [8]:
QUESTION = "Tell me about the US Economy"

answer_gemini = answer_question(QUESTION)
display("User Question:")
display(QUESTION)
display("--------------------------------")
display("Gemini Answer:")
display(answer_gemini)

'Retrieved Data:'

'The US economy has grown more than expected, expanding at an annual rate of 3.8% in the last quarter of 2004.\n\nThe gross domestic product figure was ahead of the 3.1% the government estimated a month ago. The rise reflects stronger spending by businesses on capital equipment and a smaller-than-expected trade deficit. GDP is a measure of a country\'s economic health, reflecting the value of the goods and services it produces.\n\nThe new GDP figure, announced by the Commerce Department on Friday, also topped the 3.5% growth rate that economists had forecast ahead of Friday\'s announcement. Growth was at an annual rate of 4% in the third quarter of 2004 and for the year it came in at 4.4%, the best figure in five years. However, the positive economic climate may lead to a rise in interest rates, with many expecting US rates to rise on 22 March. In the January-to-March quarter, the economy is expected to grow at an annual rate of about 4%, economists forecast. In the final quarter of 20

' . . . '

'User Question:'

'Tell me about the US Economy'

'--------------------------------'

'Gemini Answer:'

"## The US Economy in 2004: A Year of Strong Growth\n\nThe US economy experienced robust growth in 2004, exceeding expectations with a **3.8% annual rate** in the final quarter. This surpassed the government's initial estimate of 3.1% and topped the 3.5% growth rate predicted by economists.\n\n**Key factors driving this growth:**\n\n* **Stronger business spending:** Businesses increased capital equipment and software spending by 18% in the final quarter, indicating investment in future growth.\n* **Smaller trade deficit:** The trade deficit was lower than expected, contributing to the overall economic expansion.\n* **Healthy consumer spending:** Consumer spending grew 4.2% in the final quarter, though slightly lower than the previous quarter's 5.1%.\n\n**Overall, 2004 saw:**\n\n* **4.4% GDP growth:** The best performance in five years.\n* **2.2 million new jobs created:** The highest number in five years.\n* **5.4% unemployment rate:** Showing signs of potential decline with consistent